In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..") 

from src.utils import *
from src.plots import *
from src.late_fusion import *

# Late fusion

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
audio_embs = torch.load('../embeddings/audio/raw_wavlm.pt')
video_embs = torch.load('../embeddings/video/raw.pt')

In [3]:
train_loader_audio, test_loader_audio = train_test_dataloader(audio_embs)

model_audio = MLP(input_dim = 1024, hidden_dims = [512], output_dim = 6).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer_audio = optim.Adam(model_audio.parameters(), lr = 0.001)
num_epochs = 50


####################################################################################################


train_loader_video, test_loader_video = train_test_dataloader(video_embs)

model_video = MLP(input_dim = 512, hidden_dims = [256], output_dim = 6).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer_video = optim.Adam(model_video.parameters(), lr = 0.001)
num_epochs = 50


####################################################################################################


PARAMS = {'models': [model_audio, model_video],
        'optimizers': [optimizer_audio, optimizer_video],
        'train_loaders': [train_loader_audio, train_loader_video],
        'test_loaders': [test_loader_audio, test_loader_video],
        'criterion': criterion,
        'num_epochs': num_epochs,
        'device': DEVICE}

/home/mweuro/venvy/best_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_late_fusion(**PARAMS)

Epoch 5/50
Val Loss Audio: 0.7547,                    Val Accuracy Audio: 0.7126,                    Val Precision Audio: 0.7349,                    Val Recall Audio: 0.7126.
Val Loss Video: 1.2802,                    Val Accuracy Video: 0.4607,                    Val Precision Video: 0.5299,                    Val Recall Video: 0.4607.
Fusion Loss: 1.5595,                    Fusion Accuracy: 0.5346,                    Fusion Precision: 0.5282,                    Fusion Recall: 0.5346.
--------------------------------------------------


Epoch 10/50
Val Loss Audio: 0.7114,                    Val Accuracy Audio: 0.7388,                    Val Precision Audio: 0.7551,                    Val Recall Audio: 0.7388.
Val Loss Video: 1.0952,                    Val Accuracy Video: 0.5588,                    Val Precision Video: 0.5733,                    Val Recall Video: 0.5588.
Fusion Loss: 1.5125,                    Fusion Accuracy: 0.5762,                    Fusion Precision: 0.5740,                    Fusion Recall: 0.5762.
--------------------------------------------------


Epoch 15/50
Val Loss Audio: 0.7678,                    Val Accuracy Audio: 0.7099,                    Val Precision Audio: 0.7321,                    Val Recall Audio: 0.7099.
Val Loss Video: 1.0592,                    Val Accuracy Video: 0.5850,                    Val Precision Video: 0.6053,                    Val Recall Video: 0.5850.
Fusion Loss: 1.4894,                    Fusion Accuracy: 0.6004,                    Fusion Precision: 0.6013,                    Fusion Recall: 0.6004.
--------------------------------------------------


Epoch 20/50
Val Loss Audio: 0.6741,                    Val Accuracy Audio: 0.7461,                    Val Precision Audio: 0.7497,                    Val Recall Audio: 0.7461.
Val Loss Video: 1.0605,                    Val Accuracy Video: 0.5675,                    Val Precision Video: 0.5567,                    Val Recall Video: 0.5675.
Fusion Loss: 1.4697,                    Fusion Accuracy: 0.6300,                    Fusion Precision: 0.6295,                    Fusion Recall: 0.6300.
--------------------------------------------------


Epoch 25/50
Val Loss Audio: 0.7650,                    Val Accuracy Audio: 0.7240,                    Val Precision Audio: 0.7368,                    Val Recall Audio: 0.7240.
Val Loss Video: 0.9359,                    Val Accuracy Video: 0.6273,                    Val Precision Video: 0.6236,                    Val Recall Video: 0.6273.
Fusion Loss: 1.4543,                    Fusion Accuracy: 0.6407,                    Fusion Precision: 0.6409,                    Fusion Recall: 0.6407.
--------------------------------------------------


...

Epoch 35/50
Val Loss Audio: 0.7579,                    Val Accuracy Audio: 0.7495,                    Val Precision Audio: 0.7568,                    Val Recall Audio: 0.7495.
Val Loss Video: 0.9026,                    Val Accuracy Video: 0.6427,                    Val Precision Video: 0.6429,                    Val Recall Video: 0.6427.
Fusion Loss: 1.4353,                    Fusion Accuracy: 0.6595,                    Fusion Precision: 0.6635,                    Fusion Recall: 0.6595.
--------------------------------------------------


Epoch 40/50
Val Loss Audio: 0.7561,                    Val Accuracy Audio: 0.7495,                    Val Precision Audio: 0.7530,                    Val Recall Audio: 0.7495.
Val Loss Video: 0.9510,                    Val Accuracy Video: 0.6279,                    Val Precision Video: 0.6461,                    Val Recall Video: 0.6279.
Fusion Loss: 1.4484,                    Fusion Accuracy: 0.6555,                    Fusion Precision: 0.6570,                    Fusion Recall: 0.6555.
--------------------------------------------------


Epoch 45/50
Val Loss Audio: 0.8361,                    Val Accuracy Audio: 0.7515,                    Val Precision Audio: 0.7583,                    Val Recall Audio: 0.7515.
Val Loss Video: 0.8760,                    Val Accuracy Video: 0.6655,                    Val Precision Video: 0.6626,                    Val Recall Video: 0.6655.
Fusion Loss: 1.4281,                    Fusion Accuracy: 0.6729,                    Fusion Precision: 0.6725,                    Fusion Recall: 0.6729.
--------------------------------------------------


Epoch 50/50
Val Loss Audio: 0.8098,                    Val Accuracy Audio: 0.7455,                    Val Precision Audio: 0.7492,                    Val Recall Audio: 0.7455.
Val Loss Video: 0.8859,                    Val Accuracy Video: 0.6488,                    Val Precision Video: 0.6622,                    Val Recall Video: 0.6488.
Fusion Loss: 1.4287,                    Fusion Accuracy: 0.6702,                    Fusion Precision: 0.6704,                    Fusion Recall: 0.6702.
--------------------------------------------------
BEST EPOCH: 44.00          BEST LOSS: 1.43          BEST ACCURACY: 0.67          BEST PRECISION: 0.67          BEST RECALL: 0.67


([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50],
 [1.6676373481750488,
  1.6263242959976196,
  1.5955266952514648,
  1.5752516984939575,
  1.5595287084579468,
  1.5459786653518677,
  1.541273832321167,
  1.5261476039886475,
  1.5180742740631104,
  1.5125393867492676,
  1.5055553913116455,
  1.4985154867172241,
  1.5001252889633179,
  1.5029629468917847,
  1.4894177913665771,
  1.4862276315689087,
  1.4810059070587158,
  1.4852458238601685,
  1.4815711975097656,
  1.469683289527893,
  1.4689165353775024,
  1.4644508361816406,
  1.4598745107650757,
  1.4593464136123657,
  1.4542771577835083,
  1.4574142694473267,
  1.461561918258667,
  1.4520293474197388,
  1.44747793674469,
  1.4521677494049072,
  1.4488413333892822,
  1.4418658018112183,
  1.44625